# Models
The majority of our implementations for for next utterance classification including:
1. completely random predictor
2. TF-IDF
3. word2vec (with averaging)
4. doc2vec
5. LDA
6. K-means with Gaussian Naive Bayes Classifier

Check the report for the locations of other models we implemented.

Evaluation using recall@k on test set with cosine similarity.

In [109]:
import urllib
import pickle
import numpy as np
import pandas as pd
from copy import copy
from gensim.models import doc2vec, word2vec
import gensim
from collections import namedtuple
import timeit
import csv

from modules.evaluation_metrics import recall_at_k
from modules import tfidf
from modules import gaussian_naive_bayes
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.preprocessing import scale

In [2]:
# Read in data
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [8]:
# Get data into right format i.e. list of strings
train_data = np.append(train.Context.values,train.Utterance.values)

In [7]:
print train.shape
train.head()

(1000000, 3)


,Context,Utterance,Label
0,i think we could import the old comment via rs...,basic each xfree86 upload will not forc user t...,1
1,i 'm not suggest all - onli the one you modifi...,sorri __eou__ i think it be ubuntu relat . __e...,0
2,afternoon all __eou__ not entir relat to warti...,"yep . __eou__ oh , okay . i wonder what happen...",0
3,interest __eou__ grub-instal work with / be ex...,that the one __eou__,1
4,and becaus python give mark a woodi __eou__ __...,( i think someon be go to make a joke about .a...,1


In [8]:
print test.shape
test.head()

(18920, 11)


,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,anyon know whi my stock oneir export env var u...,nice thank ! __eou__,"wrong channel for it , but check efnet.org , u...","everi time the kernel chang , you will lose vi...",ok __eou__,! nomodeset > acer __eou__ i 'm assum it be a ...,http : //www.ubuntu.com/project/about-ubuntu/d...,thx __eou__ unfortun the program be n't instal...,how can i check ? by do a recoveri for test ? ...,my humbl apolog __eou__,# ubuntu-offtop __eou__
1,i set up my hd such that i have to type a pass...,"so you dont know , ok , anyon els ? __eou__ yo...","nmap be nice , but it be n't what i be look fo...",ok __eou__,cdrom work fine on window . __eou__ i dont thi...,"ah yes , i have read return as rerun __eou__",hm ? __eou__,"not the case , lts be everi other .04 releas ....",pretti much __eou__,i use the one i download from amd __eou__,"ffmpeg be part of the packag , quixotedon , at..."
2,im tri to use ubuntu on my macbook pro retina ...,just wonder how it run __eou__,"yes , that 's what i do , export it to a `` id...",noth - i be talk about the question of myhero ...,that should fix the font be too larg __eou__,"okay , so hcitool echo back hci0 < mac address...",i get to the menu with option such as tri ubun...,whi do u need analyz __eou__ it be a toy __eou...,cntrl-c may stop the command but it doe n't fi...,"if you re onli go to run ubuntu , just get a n...",the one which be not pick up at the moment be ...
3,no suggest ? __eou__ link ? __eou__ how can i ...,you cant load anyth via usb or cd when luk be ...,-p sorri ... __eou__ nmap -p22 __eou__ it doe ...,i guess so i ca n't even launch it . __eou__,note __eou__,rxvt-unicod be one __eou__,i tar all of ~ __eou__,i tar all of ~ __eou__,"i do n't realli know if i can help , but i be ...","that work just fine , thank ! __eou__",thank you __eou__
4,i just ad a second usb printer but not sure wh...,i be set it up under the printer configur __eo...,i 'd say the most common venu would be via lau...,"the old hardi man page , http : //manpages.ubu...",i ll give a tri __eou__,"by the way , the url you post for davf be from...",http : //ubuntuforums.org/showthread.php ? t=1...,"so i load up putti gui , then what do i do ? _...","you should read error messag , it say be you r...",wait the colleg semest to close just to make s...,i be call myself a jerk . all i know be that y...


## Completely Random Predictor
This naive predictor randomly picks one of the responses for each of the rows in the test data set. Therefore, we would expect it to have ~10% accuracy with recall@1, ~20% accuracy with recall@2, ~50% accuracy with recall@5 and 100% accuracy with recall@10.

In [4]:
# Completely random prediction model
def random_predictor(context, test):
    return np.random.choice(len(test), 10, replace=False)

In [5]:
# As a sanity check, let's see if the random predictor performs as expected
y_random = [random_predictor(test.Context[x], test.iloc[x,1:].values) for x in range(len(test))]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y_random, k)))

Recall @ 1, 10 total choices: 0.0974101
Recall @ 2, 10 total choices: 0.195032
Recall @ 5, 10 total choices: 0.493869
Recall @ 10, 10 total choices: 1


The random predictor does indeed perform as expected.

## TF-IDF Weighting
Now let's try to use **TF-IDF** weighting to vectorize the contexts and responses and use **cosine similarity** to compute the rank each of the possible responses.

In [ ]:
# Fit TF-IDF model
tfidf_model = tfidf.TFIDF_Predictor()
tfidf_model.train(train_data)

In [10]:
# Evaluate model performance
y = [tfidf_model.predict(test.Context[x], test.iloc[x,1:].values) for x in range(test.shape[0])]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.495032
Recall @ 2, 10 total choices: 0.596882
Recall @ 5, 10 total choices: 0.766121
Recall @ 10, 10 total choices: 1


It's a basic method, but still a robust one and performs respectably.

## word2vec
Next let's generate word embeddings using word2vec and average the word2vec representations for each word in a message in order to get a feature vector for that message. 

In [153]:
# Split each of the messages into lists of words
train_data_lists = [m.split() for m in train_data]

In [ ]:
# Train w2v model
w2v_model = word2vec.Word2Vec(train_data_lists, size=100, min_count=3, iter=3, workers=3)

In [17]:
# Save model
w2v_model.save('w2v_models/size100_mincount3_iter3')

In [36]:
# Train w2v model of size 200, min_count 5 and 20 iterations
start = timeit.default_timer()
w2v_model2 = word2vec.Word2Vec(train_data_lists, size=200, min_count=5, iter=20, workers=4)

# Save model
w2v_model2.save('w2v_models/size200_mincount5_iter20')
stop = timeit.default_timer()
print stop - start 

1899.75511289


In [171]:
# Load trained w2v model
w2v_model = word2vec.Word2Vec.load('w2v_models/size200_mincount5_iter20')

In [177]:
def average_w2v(document, model, num_features):
    '''
    Calculate a feature vector for a document by averaging the 
    word2vec representations of its constituent words
    
    Args:
        document: a message as a str
        model: trained w2v model
        num_features: length of feature vector
        
    Returns:
        Feature vector that is the average of the word2vec representations of all words in the document
    '''
    # Get vocab of the model
    vocab = set(model.index2word)
    vector = np.zeros(num_features)
    num_words = 0
    # Split document into list of words
    words = document.split()
    
    # If word exists in vocabulary, add its w2v to sum
    for word in words:
        if word in vocab:
            vector = np.add(vector, model[word])
            num_words = num_words + 1
        
    # Average by the number of words
    vector = np.divide(vector, num_words)
    
    return vector

In [178]:
def w2v_predict(model, context, responses, size):
    '''
    Calculates the cosine similarity between the context and each of the possible responses
    
    Args:
        model: a word2vec model
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        size: size of word2vec vectors for the model
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    context_vec = average_w2v(context, model, size)
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the averaged word2vec vector of response and context
        response_vec = average_w2v(response, model, size)
        sim = cosine_similarity(context_vec.reshape(1, -1), response_vec.reshape(1, -1))[0][0]
        sims.append(sim)
    
    return np.argsort(sims, axis=0)[::-1]

In [181]:
# Evaluate w2v averaging model's performance
y = [w2v_predict(w2v_model, test.Context[x], test.iloc[x,1:].values, 200) for x in range(len(test))]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.310307
Recall @ 2, 10 total choices: 0.445983
Recall @ 5, 10 total choices: 0.70333
Recall @ 10, 10 total choices: 1


## doc2vec
Doc2vec generates vector embeddings for entire documents, so let's use those embeddings as feature vectors.

In [4]:
taggedMessage = namedtuple('TaggedMessage', 'words tags')
documents = []

# Preprocess messages
for i, message in enumerate(train_data):
    # Split into lists of words
    words = message.split()
    # Add a tag corresponding to the index of the message
    tags = [i]
    x = taggedMessage(words, tags)
    documents.append(taggedMessage(words, tags))

In [7]:
# Train doc2vec model
start = timeit.default_timer()
d2v = doc2vec.Doc2Vec(documents, size=100, workers=4, iter=5)
stop = timeit.default_timer()
print stop - start 

1040.17123508


In [8]:
d2v.save("d2v_size100_iter5")

In [9]:
# Train doc2vec model with different parameters
start = timeit.default_timer()
d2v2 = doc2vec.Doc2Vec(documents, size=200, workers=4, iter=20)
stop = timeit.default_timer()
print stop - start 

5021.152601


In [10]:
d2v2.save("d2v_size200_iter20")

In [15]:
def d2v_predict(model, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses
    
    Args:
        model: a doc2vec model
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    context_vector = model.infer_vector(context.split())
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the doc2vec vectors of response and context
        response_vector = model.infer_vector(response.split())
        sim = cosine_similarity(context_vector.reshape(1, -1), response_vector.reshape(1, -1))[0][0]
        sims.append(sim)
    
    return np.argsort(sims, axis=0)[::-1]

In [17]:
# Evaluate model performance with doc2vec model with size 200, 20 iterations
y = [d2v_predict(d2v2, test.Context[x], test.iloc[x,1:].values) for x in range(test.shape[0])]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.333245
Recall @ 2, 10 total choices: 0.477378
Recall @ 5, 10 total choices: 0.737474
Recall @ 10, 10 total choices: 1


Our best doc2vec model performs slightly worse than TF-IDF, though slightly better than word2vec with averaging.

In [18]:
# Evaluate model performance with doc2vec model with size 100, 5 iterations
y = [d2v_predict(d2v, test.Context[x], test.iloc[x,1:].values) for x in range(test.shape[0])]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.280391
Recall @ 2, 10 total choices: 0.423044
Recall @ 5, 10 total choices: 0.70222
Recall @ 10, 10 total choices: 1


As expected, the doc2vec model which took less training time performs more poorly.

## Export best doc2vec model to tsv
Next let's export our best doc2vec model's vectors to tsv for visualization with Google's embedding projector (http://projector.tensorflow.org/)

In [57]:
# Write a tsv file with one doc2vec per row
with open('tsv/doc2vec_first10000.tsv', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
#     for v in d2v2.docvecs:
#         writer.writerow(v)
    # Take a subset for now
    for i in range(10000):
        writer.writerow(d2v2.docvecs[i])

In [58]:
# Write tsv file with metadata (i.e. all the training text, one per line)
with open('tsv/metadata_first10000.tsv', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    for doc in train_data[:10000]:
        writer.writerow([doc])

## LDA
LDA is a well known topic modelling algorithm from which you can learn the topic distributions of a corpus and infer the topic distribution for a new text.

In [38]:
# Generate corpus for use with LDA
texts = [doc.split() for doc in train_data]

In [40]:
# Create dictionary for corpus
dictionary = gensim.corpora.Dictionary(texts)
# Save dictionary for use later
dictionary.save('dict/ubuntu.dict')

In [ ]:
# Create corpus of bag of words
corpus = [dictionary.doc2bow(text) for text in texts]
gensim.corpora.MmCorpus.serialize('/corpora/ubuntu_bow.mm', corpus)  # store to disk, for later use

In [ ]:
start = timeit.default_timer()
# Train LDA model
lda = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics=100, workers=4)
stop = timeit.default_timer()
print stop - start 

In [97]:
# Load trained LDA model
lda = gensim.models.LdaModel.load("LDA/lda_model1")

In [98]:
# load saved dictionary
dictionary = gensim.corpora.dictionary.Dictionary.load("dict/ubuntu.dict")

In [99]:
def get_lda_feature_vector(topic_dist, num_topics):
    '''
    Convert gensim LDA topic distribution for a given document to feature vector of length number of topics
    
    Args:
        topic_dist: topic distribution for a message from gensim LDA model
                    a list of (topic index, proportion) pairs
                    i.e. [(5, 0.59), (12, 0.11)...]
        num_topics: total number of topics in the model, for use as length of feature vector
            
    Returns:
        Feature vector for document where the index corresponds to topic number and value is proportion for that topic
    '''
    vector = np.zeros(num_topics)
    
    # Fill in values for nonzero topics
    for index, value in topic_dist:
        vector[index] = value
            
    return vector

In [91]:
def lda_predict(model, dictionary, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses,
    returning a ranked list of responses sorted in decreasing order by cosine similarity
    
    Args:
        model: a lda model
        dictionary: dictionary associated with the lda model
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    # Infer topic distribution and vectorize
    context_vector = get_lda_feature_vector(model[dictionary.doc2bow(context.split())], model.num_topics)
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the lda feature vectors of response and context
        response_vector = get_lda_feature_vector(model[dictionary.doc2bow(response.split())], model.num_topics)
        sim = cosine_similarity(context_vector.reshape(1, -1), response_vector.reshape(1, -1))[0][0]
        sims.append(sim)
        
    return np.argsort(sims, axis=0)[::-1]

In [92]:
# Evaluate model performance with lda model
y = [lda_predict(lda, dictionary, test.Context[x], test.iloc[x,1:].values) for x in range(len(test.shape[0]))]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.309
Recall @ 2, 10 total choices: 0.451
Recall @ 5, 10 total choices: 0.58
Recall @ 10, 10 total choices: 1
5038.85787487


LDA does not perform particularly well, though this may have to do with the parameters with which we trained it. Unfortunately LDA takes a long time to train on this volume of data, so it's difficult to tune.

## K-Means Clustering and Naive Bayes
Our main custom algorithm focusing on techniques covered in the class. First we cluster the doc2vec vectorizations of each text using minibatch k-means. Then we train a naive bayes classifier using the clustering for class labels. We generate feature vectors for new texts using the posterior probabilities for each cluster based on Naive Bayes. 

One important note to make is that we use a Gaussian Naive Bayes classifier, as opposed to the multinomial one we implemented in class, since we are dealing with doc2vec embeddings as opposed to bag of words representation. The latter can be modeled with a multinomial distribution since it consists of count data, whereas a Gaussian is a better approximation for the former since we are dealing with continuous variables.

In [3]:
# Load trained doc2vec model
d2v2 = doc2vec.Doc2Vec.load("d2v_models/d2v_size200_iter20")

In [4]:
# Create array of doc2vec vectors
X = []
for v in d2v2.docvecs:
    X.append(v)

X = np.array(X)

In [8]:
# Cluster messages using minibatch k-means
mbkmeans = MiniBatchKMeans(n_clusters=100).fit(X)

In [108]:
mbkmeans.counts_

array([ 1811, 15893,  4178,   933, 11971, 12963, 26825, 16606, 10324,
       16300,  2098,  4099, 31382, 17287, 21533,  6242,  2173, 24888,
       10882, 10612,  2593, 10676,  1824, 14499, 24770, 10952,  9274,
       19802,  1068, 70106,   681,  9707, 17695, 13869,  7871,  3073,
       15131, 11157, 11953, 16035,  7113, 21120,   681, 14956,  3333,
       12615,   702, 22298, 22444, 23566,  1169,  8595,  4381, 19173,
         779, 16844,  3973, 21909,  2456,  5122, 21109, 20738,  9466,
       17923,  1056,  1130, 18391,  3530,  7408, 12595,  3999,  6580,
        5839, 13843,  8221,  1314, 17344,   685,  9082,  9658, 18850,
         681,   681, 28090,  7782,  6678, 11144, 19752, 31288,  4871,
       23209, 16423, 15197, 15355, 11027,   683, 15253,  3947, 14790, 18768], dtype=int32)

In [95]:
def nb_predict(nb_model, d2v_model, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses
    
    Args:
        nb_model: a trained naive bayes classifier
        count_vectorizer: a fitted bow vectorizer
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    # Get vector of probabilities for each of the clusters for the context
    context_vector = nb_model.predict_proba(d2v_model.infer_vector(context.split()).reshape(1, -1))
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the inferred doc2vec vectors of response and context
        response_vector = nb_model.predict_proba(d2v_model.infer_vector(response.split()).reshape(1, -1))
        sim = cosine_similarity(context_vector, response_vector)[0][0]
        sims.append(sim)
            
    return np.argsort(sims, axis=0)[::-1]

In [73]:
# Train Gaussian naive bayes classifier
gaussian_nb = GaussianNB()
gaussian_nb.fit(X, mbkmeans.labels_)

GaussianNB(priors=None)

In [96]:
# Evaluate model performance with clustering + gaussian naive bayes approach
y = [nb_predict(gaussian_nb, d2v2, test.Context[x], test.iloc[x,1:].values) for x in range(len(test))]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.161522
Recall @ 2, 10 total choices: 0.265803
Recall @ 5, 10 total choices: 0.52648
Recall @ 10, 10 total choices: 1


This model unfortunately does not perform particularly well.

## Using our own implementation of Gaussian Naive Bayes
We made our own implementation of Gaussian Naive Bayes which trained in a reasonable amount of time, demonstrating the speed of training Naive Bayes, even on large datasets like this one. Unfortunately, we ran into some time complexity issues when it came to evaluation, so we weren't able to validate on the full evaluation set. Had we more time, we definitely would have looked into optimizing our implementaion.

In [115]:
# Format data for use in our implementation of Naive Bayes
train_df = pd.DataFrame(X, columns=list(range(X.shape[1])))
labels_df = pd.DataFrame(mbkmeans.labels_, columns=["labels"])
train_df = pd.concat([labels_df, train_df], axis=1)

In [116]:
train_df.head()

,labels,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,90,-0.134405,-0.085782,0.257740,0.118339,-0.083452,0.081222,0.010578,-0.096038,0.014784,...,-0.211393,-0.188452,0.166451,0.004803,0.020031,-0.111762,-0.093503,0.074719,-0.033617,0.075097
1,49,0.060166,-0.082015,0.152485,0.130863,-0.067056,0.197265,-0.096574,-0.071066,-0.013178,...,0.021187,0.036158,0.034269,-0.057416,-0.029969,-0.067793,-0.168198,0.080696,0.045063,0.027310
2,90,-0.000282,0.077069,0.063212,-0.021065,0.046702,0.037404,-0.003899,0.017025,-0.054937,...,-0.060134,-0.080793,-0.110839,-0.033480,0.003020,-0.082577,0.075278,-0.061084,-0.060006,-0.084176
3,94,0.178126,-0.049867,0.162412,-0.021864,0.100686,-0.020342,0.024187,0.025997,0.163804,...,-0.041363,-0.129095,0.149627,-0.086052,-0.054755,-0.100571,0.018280,-0.231799,0.028404,-0.017776
4,10,-0.027928,0.155952,0.017260,0.148459,-0.030822,0.095529,-0.022044,-0.173755,-0.071279,...,0.011883,0.045367,0.040716,-0.040944,-0.026997,0.171865,-0.009010,0.083316,0.034317,0.036460


In [118]:
# Train our implementation of a gaussian naive bayes classifier
our_gaussian_nb = gaussian_naive_bayes.GaussianNaiveBayes(train_df, 100)

In [147]:
def our_nb_predict(nb_model, d2v_model, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses using our implementation of Naive Bayes
    
    Args:
        nb_model: a trained naive bayes classifier
        count_vectorizer: a fitted bow vectorizer
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    # Get vector of probabilities for each of the clusters for the context
    context_vector = nb_model.predict_proba(d2v_model.infer_vector(context.split()))
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the inferred doc2vec vectors of response and context
        response_vector = nb_model.predict_proba(d2v_model.infer_vector(response.split()))
        sim = cosine_similarity(context_vector.reshape(1,-1), response_vector.reshape(1,-1))[0][0]
        sims.append(sim)
            
    return np.argsort(sims, axis=0)[::-1]

In [148]:
# Evaluate model performance with clustering + our implementation of gaussian naive bayes
start = timeit.default_timer()
y = [our_nb_predict(our_gaussian_nb, d2v2, test.Context[x], test.iloc[x,1:].values) for x in range(10)]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))
stop = timeit.default_timer()
print stop - start

Recall @ 1, 10 total choices: 0.1
Recall @ 2, 10 total choices: 0.2
Recall @ 5, 10 total choices: 0.7
Recall @ 10, 10 total choices: 1
706.754172087


In [149]:
# Evaluate model performance with clustering + our implementation of gaussian naive bayes
start = timeit.default_timer()
y = [our_nb_predict(our_gaussian_nb, d2v2, test.Context[x], test.iloc[x,1:].values) for x in range(100)]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))
stop = timeit.default_timer()
print stop - start

Recall @ 1, 10 total choices: 0.21
Recall @ 2, 10 total choices: 0.3
Recall @ 5, 10 total choices: 0.56
Recall @ 10, 10 total choices: 1
7050.78050494
